## Import Libraries

In [1]:
# importing the necessary modules
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import os
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


## Data Import

In [ ]:
#Set variable for page to be opened and url to be concatenated 
page =requests.get('https://www.opm.gov/data/index.aspx')
baseurl= 'https://www.opm.gov/'
#Use BeautifulSoup to clean up the page
soup = BeautifulSoup(page.text, "html.parser")
soup.prettify()

# Creating a new file to store the zip file links
newfile = open('zipfiles.txt','w')
#Find all the links on the page that end in .zip and write them into the text file
for anchor in soup.findAll('a', href=True):
    links = anchor['href']
    if links.endswith('.zip'):
        newfile.write(links + '\n')
newfile.close()

## Automate zipfile download and extract

In [ ]:
#Fetching the links for the zip file and downloading the files
with open('zipfiles.txt', 'r') as links:
    count = 0
    for link in links:
        if count == 2:
            break
        if link:
            filename1= link.split('/')[-1]
            filename= filename1[:-1]
            link = baseurl + link
            print(filename + ' file started to download')
            response = requests.get(link[:-1])
            
            # Writing the zip file into local file system
            with open(filename,'wb') as output_file:
                output_file.write(response.content)
            print(filename + ' file is downloaded')
            with ZipFile(filename, 'r') as zip:
                # printing all the contents of the zip file
                #zip.printdir()
                print('')
            z = ZipFile(BytesIO(response.content))
            # gets September data
            if count == 0:
                def new_directory(directory, filename):
                  # Before creating a new directory, check to see if it already exists
                  if os.path.isdir(directory) == False:
                    os.mkdir(directory)
                #print(directory+" "+filename)
                z.extractall('SepData')
                print('September data Extracted')
                print('-------------------------------------------------------------')
            # gets June data    
            elif count == 1:
                def new_directory(directory, filename):
                  # Before creating a new directory, check to see if it already exists
                  if os.path.isdir(directory) == False:
                    os.mkdir('JunData')
                z.extractall('JunData')
                print('June data Extracted')
            count += 1
            

### Merge the datasets

In [2]:
# Importing September data using pandas
FACTDATA_S = pd.read_csv("/Users/suryamanitejendla/Downloads/DAEN-690-main/FedScope Employment Cube (September 2021)/FACTDATA_SEP2021.TXT")
# Importing June data using pandas
FACTDATA_J = pd.read_csv("/Users/suryamanitejendla/Downloads/DAEN-690-main/FedScope Employment Cube (June 2021)/FACTDATA_JUN2021.TXT")
#merge data of both Sep and June using pandas
data = pd.concat([FACTDATA_S, FACTDATA_J], axis=0)
data.head()

/Users/suryamanitejendla/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,4,6,8,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AA00,11,F,13,NaN,F,0340,2,ES,ES-**,S,XXXX,2,50,F,1,202109,1,192500.0,18.3
1,AA00,11,I,15,NaN,H,0905,1,ES,ES-**,R,XXXX,2,50,F,1,202109,1,187000.0,28.7
2,AA00,11,E,15,15.0,E,0905,1,99,GS-15,Q,XXXX,2,30,F,1,202109,1,172500.0,12.0
3,AA00,11,E,15,15.0,E,0905,1,99,GS-15,N,XXXX,8,30,F,1,202109,1,144128.0,10.0
4,AA00,11,F,15,13.0,E,0905,1,99,GS-13,K,XXXX,8,30,F,1,202109,1,114059.0,12.9


# Data Cleaning

In [ ]:
# Finding the type of each variable in the dataset
#data.dtypes

### Missing Values

In [ ]:
# Sum of null values in each Attribute
#data.isnull().sum()

### Find null values grouped by agency (No Hard coding)

In [3]:
# Find null values grouped by agency
dflist1 = data.columns
# creating a new column with sum of null values in each row
dn = data.groupby(['AGYSUB'])[dflist1].apply(lambda x: x.isnull().sum())
dn['results'] = dn.sum(axis = 1)
dn.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS,results
AGYSUB,,,,,,,,,,,,,,,,,,,,,
AA00,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
AB00,0,0,0,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
AF02,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,20,0,22
AF03,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,6,0,8
AF06,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,7


## List of Agencies with zero null values (No Hard coding)

In [4]:
# Finding agencies that has zero missing values
dzero = dn[dn['results'] == 0]
dzero.iloc[: , 1:-1] #trim first and last columns of the dataframe 1=AGYSUB, -1=results
# Fetching only the agencies with zero null values
zero_NA_agy = dzero.iloc[:,0]
less_na_agy = zero_NA_agy.keys().tolist()
#less_na_agy
# verifying the zero null agencies with main data agencies and printing all columns
data_na_agy = data[data['AGYSUB'].isin(less_na_agy)]
data_na_agy

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
38686,AF0U,24,K,08,13.0,D,2210,2,99,GG-13,L,2210,8,38,F,1,202109,1,127885.0,8.4
134010,AF1Q,40,J,10,13.0,G,1670,2,99,GS-13,K,XXXX,8,10,F,1,202109,1,116715.0,20.4
134011,AF1Q,40,E,13,12.0,E,0560,2,99,GS-12,I,XXXX,8,10,F,1,202109,1,92988.0,14.3
134012,AF1Q,40,K,04,13.0,J,2152,2,99,GS-13,L,XXXX,8,10,F,1,202109,1,129136.0,37.3
134013,AF1Q,40,I,04,12.0,F,1670,2,99,GS-12,I,XXXX,8,10,F,1,202109,1,98155.0,17.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1779178,VAKB,11,D,17,11.0,B,343,2,99,GS-11,G,XXXX,8,15,F,1,202106,1,72750.0,1.7
1779179,VAKB,11,D,17,11.0,B,343,2,99,GS-11,G,XXXX,8,15,F,1,202106,1,72750.0,1.3
1779180,VAKB,11,E,13,12.0,A,301,2,99,GS-12,H,XXXX,8,15,F,1,202106,1,87198.0,0.8
1779181,VAKB,11,E,17,11.0,A,301,2,99,GS-11,G,XXXX,8,20,F,2,202106,1,72750.0,0.8


## Comparing both Dataframes

In [49]:
data_june = data_na_agy[(data_na_agy["DATECODE"]) == 202106]
data_sept = data_na_agy[(data_na_agy["DATECODE"]) == 202109]

In [50]:
cols = data_june.columns.values.tolist();

In [51]:
cols.remove("LOS")

In [52]:
cols.remove("DATECODE")

In [53]:
#cols.remove("SALARY")

In [54]:
commonPeople = pd.merge(data_june, data_sept, on=cols, how='inner')

In [57]:
commonPeople

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE_x,EMPLOYMENT,SALARY,LOS_x,DATECODE_y,LOS_y
0,AF1Q,40,J,10,13.0,G,1670,2,99,GS-13,...,8,10,F,1,202106,1,116715.0,20.2,202109,20.4
1,AF1Q,40,E,13,12.0,E,0560,2,99,GS-12,...,8,10,F,1,202106,1,92988.0,14.1,202109,14.3
2,AF1Q,40,K,04,13.0,J,2152,2,99,GS-13,...,8,10,F,1,202106,1,129136.0,40.9,202109,37.3
3,AF1Q,40,I,04,12.0,F,1670,2,99,GS-12,...,8,10,F,1,202106,1,98155.0,17.6,202109,17.8
4,AF1Q,40,G,07,12.0,G,2210,2,99,GS-12,...,8,10,F,1,202106,1,95572.0,23.6,202109,23.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,VAKB,11,F,13,15.0,G,343,2,99,GS-15,...,8,15,F,1,202106,1,148932.0,20.9,202109,21.2
2733,VAKB,11,I,13,15.0,D,301,2,99,GS-15,...,8,10,F,1,202106,1,158541.0,7.1,202109,7.3
2734,VAKB,11,D,13,14.0,D,301,2,99,GS-14,...,8,10,F,1,202106,1,134782.0,6.1,202109,6.3
2735,VAKB,11,H,13,14.0,D,301,2,99,GS-14,...,8,10,F,1,202106,1,134782.0,9.6,202109,9.8


In [58]:
LOS_variant_people = commonPeople.loc[(commonPeople["LOS_x"]) < (commonPeople["LOS_y"])] 
# Filtering people who's LOS in June is greater than LOS in september

In [59]:
LOS_variant_people

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE_x,EMPLOYMENT,SALARY,LOS_x,DATECODE_y,LOS_y
0,AF1Q,40,J,10,13.0,G,1670,2,99,GS-13,...,8,10,F,1,202106,1,116715.0,20.2,202109,20.4
1,AF1Q,40,E,13,12.0,E,0560,2,99,GS-12,...,8,10,F,1,202106,1,92988.0,14.1,202109,14.3
3,AF1Q,40,I,04,12.0,F,1670,2,99,GS-12,...,8,10,F,1,202106,1,98155.0,17.6,202109,17.8
4,AF1Q,40,G,07,12.0,G,2210,2,99,GS-12,...,8,10,F,1,202106,1,95572.0,23.6,202109,23.8
5,AF1Q,40,J,04,12.0,F,1670,2,99,GS-12,...,8,10,F,1,202106,1,98155.0,17.3,202109,17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,VAKB,11,F,13,15.0,G,343,2,99,GS-15,...,8,15,F,1,202106,1,148932.0,20.9,202109,21.2
2733,VAKB,11,I,13,15.0,D,301,2,99,GS-15,...,8,10,F,1,202106,1,158541.0,7.1,202109,7.3
2734,VAKB,11,D,13,14.0,D,301,2,99,GS-14,...,8,10,F,1,202106,1,134782.0,6.1,202109,6.3
2735,VAKB,11,H,13,14.0,D,301,2,99,GS-14,...,8,10,F,1,202106,1,134782.0,9.6,202109,9.8


In [60]:
Duplicates_dropped= LOS_variant_people.drop_duplicates()

In [61]:
Duplicates_dropped

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE_x,EMPLOYMENT,SALARY,LOS_x,DATECODE_y,LOS_y
0,AF1Q,40,J,10,13.0,G,1670,2,99,GS-13,...,8,10,F,1,202106,1,116715.0,20.2,202109,20.4
1,AF1Q,40,E,13,12.0,E,0560,2,99,GS-12,...,8,10,F,1,202106,1,92988.0,14.1,202109,14.3
3,AF1Q,40,I,04,12.0,F,1670,2,99,GS-12,...,8,10,F,1,202106,1,98155.0,17.6,202109,17.8
4,AF1Q,40,G,07,12.0,G,2210,2,99,GS-12,...,8,10,F,1,202106,1,95572.0,23.6,202109,23.8
5,AF1Q,40,J,04,12.0,F,1670,2,99,GS-12,...,8,10,F,1,202106,1,98155.0,17.3,202109,17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,VAKB,11,F,13,15.0,G,343,2,99,GS-15,...,8,15,F,1,202106,1,148932.0,20.9,202109,21.2
2733,VAKB,11,I,13,15.0,D,301,2,99,GS-15,...,8,10,F,1,202106,1,158541.0,7.1,202109,7.3
2734,VAKB,11,D,13,14.0,D,301,2,99,GS-14,...,8,10,F,1,202106,1,134782.0,6.1,202109,6.3
2735,VAKB,11,H,13,14.0,D,301,2,99,GS-14,...,8,10,F,1,202106,1,134782.0,9.6,202109,9.8
